<div class="alert alert-block alert-warning">

In order to use Llama2, you need access from Meta. We suggest following the steps in this [article](https://medium.com/@yashambekar1804/getting-started-with-llma2-accessing-llama2-70b-model-and-obtaining-hugging-face-api-token-and-8773e19bda26). 

# HR Chatbot using Retrieval Augmented Generation

Many companies aim to make HR documents more accessible to employees and contractors. A recent promising approach is building a RAG system on top of HR documents. This is because it
- provides a unique response to any query
- reduces hallucinations as the answer has to be grounded in the retrieved context
- makes the process highly scalable due to automatic question-answering

However, this solution possesses its unique challenges. For example
- keeping the knowledgebase consistent and up-to-date
- run LLMs efficiently
- ensuring the generated results are correct and aligned with company communication guidelines
- etc.

In our case, we have two HR policy sources for our hypothetical company.
- An older HR policy, which contains our maternal leave policy and details on manager responsibilities
- A newer policy document that contains updated information about management responsibilities - alongside some other HR policy information

A good system will be able to:
- provide relevant information on maternity leave (only covered in the old documents)
- synthesize contradicting information and only present to us the correct documents

Regarding synthesizing information, there are contradictions between the two documents regarding management responsibilities. We are going to use the creation date as a proxy to know for two similar documents with 
- very similar information, but
- different wording, and
- slightly altered in one piece of important information
which one is more relevant to our query.

## Boilerplate

### Installation

In [1]:
%pip install superlinked==5.5.1
%pip install -U jupyter ipywidgets
%pip install -U torch transformers==4.37 accelerate

### Imports and constants

In [2]:
from datetime import timedelta, datetime

import altair as alt
import pandas as pd
import requests
import torch
from transformers import AutoTokenizer, pipeline

from superlinked.framework.common.dag.context import CONTEXT_COMMON, CONTEXT_COMMON_NOW
from superlinked.framework.common.dag.period_time import PeriodTime
from superlinked.framework.common.schema.schema import schema
from superlinked.framework.common.schema.schema_object import String, Timestamp
from superlinked.framework.common.schema.id_schema_object import IdField
from superlinked.framework.common.parser.dataframe_parser import DataFrameParser
from superlinked.framework.dsl.executor.in_memory.in_memory_executor import (
    InMemoryExecutor,
)
from superlinked.framework.dsl.index.index import Index
from superlinked.framework.dsl.query.param import Param
from superlinked.framework.dsl.query.query import Query
from superlinked.framework.dsl.query.result import Result
from superlinked.framework.dsl.source.in_memory_source import InMemorySource
from superlinked.framework.dsl.space.text_similarity_space import (
    TextSimilaritySpace,
    chunk,
)
from superlinked.framework.dsl.space.recency_space import RecencySpace

alt.renderers.enable("mimetype")
alt.data_transformers.disable_max_rows()
pd.set_option("display.max_colwidth", 1000)
START_OF_2024_TS = int(datetime(2024, 1, 2).timestamp())
EXECUTOR_DATA = {CONTEXT_COMMON: {CONTEXT_COMMON_NOW: START_OF_2024_TS}}
TOP_N = 10

## Load data

In [3]:
# the new HR policy document
r_new = requests.get(
    "https://storage.googleapis.com/superlinked-notebook-hr-knowledgebase/hr_rag_knowledgebase.txt"
)
r_new.encoding = "utf-8-sig"
text_new = r_new.text.replace("\r\n", "\n").split("\n")
r_old = requests.get(
    "https://storage.googleapis.com/superlinked-notebook-hr-knowledgebase/hr_rag_old_text.txt"
)
r_old.encoding = "utf-8-sig"
text_old = r_old.text.replace("\r\n", "\n").split("\n")

In [4]:
text_df = pd.DataFrame(text_new + text_old, columns=["body"]).reset_index()
# add timestamps to diffentiate the two sources
text_df["creation_date"] = [int(datetime(2024, 1, 1).timestamp())] * len(text_new) + [
    int(datetime(2023, 1, 1).timestamp())
] * len(text_old)
text_df.head()

,index,body,creation_date
0,0,"To ensure that employees and contractors meet security requirements, understand their responsibilities, and are suitable for their roles.",1704063600
1,1,"This policy applies to all employees of <Company Name>, consultants, contractors and other third-party entities with access to <Company Name> production networks and system resources.",1704063600
2,2,"Background verification checks on <Company Name> personnel shall be carried out in accordance with relevant laws, regulations, and shall be proportional to the business requirements, the classification of the information to be accessed, and the perceived risks. Background screening shall include criminal history checks unless prohibited by local statute. All third-parties with technical privileged or administrative access to <Company Name> production systems or networks are subject to a background check or requirement to provide evidence of an acceptable background, based on their level of access and the perceived risk to <Company Name>.",1704063600
3,3,"The skills and competence of employees and contractors shall be assessed by human resources staff and the hiring manager or his or her designees as part of the hiring process. Required skills and competencies shall be listed in job descriptions and requisitions, and/or aligned with the responsibilities outlined in the Roles and Responsibilities Policy. Competency evaluations may include reference checks, education and certification verifications, technical testing, and interviews.",1704063600
4,4,"All <Company Name> employees will undergo an annual performance review which will include an assessment of job performance, competence in the role, adherence to company policies and code of conduct, and achievement of role-specific objectives.",1704063600


## Set up Superlinked for Retrieval

In [5]:
# typed schema to describe our inputs
@schema
class ParagraphSchema:
    body: String
    created_at: Timestamp
    id: IdField

In [6]:
paragraph = ParagraphSchema()

In [7]:
# relevance space will encode our knowledgebase utilizing chunking to control the granularity of information
relevance_space = TextSimilaritySpace(
    text=chunk(paragraph.body, chunk_size=100, chunk_overlap=20),
    model="sentence-transformers/all-mpnet-base-v2",
)
# recency has a periodtime to differentiate between the document created at the beginning of this year and last year
recency_space = RecencySpace(
    timestamp=paragraph.created_at,
    period_time_list=[PeriodTime(timedelta(days=300))],
    negative_filter=-0.25,
)

In [8]:
paragraph_index = Index([relevance_space, recency_space])
paragraph_parser = DataFrameParser(
    paragraph, mapping={paragraph.id: "index", paragraph.created_at: "creation_date"}
)
source: InMemorySource = InMemorySource(paragraph, parser=paragraph_parser)
executor = InMemoryExecutor(
    sources=[source], indices=[paragraph_index], context_data=EXECUTOR_DATA
)
app = executor.run()
source.put([text_df])

In [9]:
# our simple query will make a search term possible, and gives us the opportunity to weight input aspects (relevance and recency against each other)
knowledgebase_query = (
    Query(
        paragraph_index,
        weights={
            relevance_space: Param("relevance_weight"),
            recency_space: Param("recency_weight"),
        },
    )
    .find(paragraph)
    .similar(relevance_space.text, Param("search_query"))
    .limit(Param("limit"))
)

## Perform retrieval

In [10]:
def present_result(result: Result):
    """A small helper function to present our query results"""
    df = result.to_pandas()
    df["year_created"] = [datetime.fromtimestamp(ts).year for ts in df["created_at"]]
    return df.drop("created_at", axis=1)

In [11]:
initial_query_text: str = "What should management monitor?"

First, let's do a simple retrieval based only on text similarity. We can see that the recency weight is set to 0, so the document creation dates are not affecting our results.

In [12]:
only_relevance_result = app.query(
    knowledgebase_query,
    relevance_weight=1,
    recency_weight=0,
    search_query=initial_query_text,
    limit=TOP_N,
)

present_result(only_relevance_result)

,body,id,year_created
0,"Management shall be responsible for ensuring that information security policies and procedures are reviewed annually, distributed and available, and that employees and contractors abide by those policies and procedures for the duration of their employment or engagement. Annual policy review shall include a review of any linked or referenced procedures, standards or guidelines.",6,2024
1,"Management is tasked with ensuring that information security policies and procedures undergo an bi-annual review, are disseminated and accessible, and that employees and contractors adhere to them throughout their tenure or contract period. The yearly policy assessment must encompass a scrutiny of any interconnected or referenced protocols, standards, or directives.",16,2023
2,"Management shall ensure that information security responsibilities are communicated to individuals, through written job descriptions, policies or some other documented method which is accurately updated and maintained. Compliance with information security policies and procedures and fulfillment of information security responsibilities shall be evaluated as part of the performance review process wherever applicable.",7,2024
3,"Management must guarantee that information security obligations are conveyed to individuals, either through written job outlines, policies, or alternative documented means that are regularly updated and managed accurately. Compliance with information security protocols and the fulfillment of related duties should form part of the performance evaluation process in all cases.",18,2023
4,"Management should factor in undue pressures and potential fraud scenarios when devising incentives and delineating roles, duties, and authorizations.",17,2023
5,"Management shall consider excessive pressures, and opportunities for fraud when establishing incentives and segregating roles, responsibilities, and authorities.",8,2024
6,"To ensure that employees and contractors meet security requirements, understand their responsibilities, and are suitable for their roles.",0,2024
7,"All <Company Name> employees will undergo an annual performance review which will include an assessment of job performance, competence in the role, adherence to company policies and code of conduct, and achievement of role-specific objectives.",4,2024
8,"Information security leaders and managers shall ensure appropriate professional development occurs to provide an understanding of current threats and trends in the security landscape. Security leaders and key stakeholders shall attend trainings, obtain and maintain relevant certifications, and maintain memberships in industry groups as appropriate.",11,2024
9,All <Company Name> employees and third-parties with administrative or privileged technical access to <Company Name> production systems and networks shall complete security awareness training at the time of hire and annually thereafter. Management shall monitor training completion and shall take appropriate steps to ensure compliance with this policy. Employees and contractors shall be aware of relevant information security and data privacy policies and procedures. The company shall ensure that personnel receive security and data privacy training appropriate to their role and data handling responsibilities.,9,2024


Let's observe the difference between elements with ids 16 and 6, respectively! These documents essentially say the same thing, except that the old document prescribes a bi-annual, while the fresh one an annual review on policies and procedures. Let's see what happens if we upweight recency!

In [13]:
mild_recency_result = app.query(
    knowledgebase_query,
    relevance_weight=1,
    recency_weight=0.15,
    search_query=initial_query_text,
    limit=TOP_N,
)

present_result(mild_recency_result)

,body,id,year_created
0,"Management shall be responsible for ensuring that information security policies and procedures are reviewed annually, distributed and available, and that employees and contractors abide by those policies and procedures for the duration of their employment or engagement. Annual policy review shall include a review of any linked or referenced procedures, standards or guidelines.",6,2024
1,"Management shall ensure that information security responsibilities are communicated to individuals, through written job descriptions, policies or some other documented method which is accurately updated and maintained. Compliance with information security policies and procedures and fulfillment of information security responsibilities shall be evaluated as part of the performance review process wherever applicable.",7,2024
2,"Management shall consider excessive pressures, and opportunities for fraud when establishing incentives and segregating roles, responsibilities, and authorities.",8,2024
3,"To ensure that employees and contractors meet security requirements, understand their responsibilities, and are suitable for their roles.",0,2024
4,"All <Company Name> employees will undergo an annual performance review which will include an assessment of job performance, competence in the role, adherence to company policies and code of conduct, and achievement of role-specific objectives.",4,2024
5,"Information security leaders and managers shall ensure appropriate professional development occurs to provide an understanding of current threats and trends in the security landscape. Security leaders and key stakeholders shall attend trainings, obtain and maintain relevant certifications, and maintain memberships in industry groups as appropriate.",11,2024
6,All <Company Name> employees and third-parties with administrative or privileged technical access to <Company Name> production systems and networks shall complete security awareness training at the time of hire and annually thereafter. Management shall monitor training completion and shall take appropriate steps to ensure compliance with this policy. Employees and contractors shall be aware of relevant information security and data privacy policies and procedures. The company shall ensure that personnel receive security and data privacy training appropriate to their role and data handling responsibilities.,9,2024
7,"In order to maintain a robust level of security awareness, the company will provide security-related updates and communications to company personnel on an on-going basis through multiple communication channels as needed.",10,2024
8,"Management is tasked with ensuring that information security policies and procedures undergo an bi-annual review, are disseminated and accessible, and that employees and contractors adhere to them throughout their tenure or contract period. The yearly policy assessment must encompass a scrutiny of any interconnected or referenced protocols, standards, or directives.",16,2023
9,"Background verification checks on <Company Name> personnel shall be carried out in accordance with relevant laws, regulations, and shall be proportional to the business requirements, the classification of the information to be accessed, and the perceived risks. Background screening shall include criminal history checks unless prohibited by local statute. All third-parties with technical privileged or administrative access to <Company Name> production systems or networks are subject to a background check or requirement to provide evidence of an acceptable background, based on their level of access and the perceived risk to <Company Name>.",2,2024


Now, document the relative position of element 6 compared to 16 improved as it received a score boost for it's recency. We can see the same dynamics here between documents with IDs 7 and 18 - doc 7 softened the requirement to "wherever applicable" from "in all cases". It is a useful feature that documents stating essentially the same can be differentiated through their freshness - let's boost that and see if we can have only recent, not outdated, information in our results.

In [14]:
normal_recency_result = app.query(
    knowledgebase_query,
    relevance_weight=1,
    recency_weight=0.25,
    search_query=initial_query_text,
    limit=TOP_N,
)

norm_recency_result_df = present_result(normal_recency_result)
norm_recency_result_df

,body,id,year_created
0,"Management shall be responsible for ensuring that information security policies and procedures are reviewed annually, distributed and available, and that employees and contractors abide by those policies and procedures for the duration of their employment or engagement. Annual policy review shall include a review of any linked or referenced procedures, standards or guidelines.",6,2024
1,"Management shall ensure that information security responsibilities are communicated to individuals, through written job descriptions, policies or some other documented method which is accurately updated and maintained. Compliance with information security policies and procedures and fulfillment of information security responsibilities shall be evaluated as part of the performance review process wherever applicable.",7,2024
2,"Management shall consider excessive pressures, and opportunities for fraud when establishing incentives and segregating roles, responsibilities, and authorities.",8,2024
3,"To ensure that employees and contractors meet security requirements, understand their responsibilities, and are suitable for their roles.",0,2024
4,"All <Company Name> employees will undergo an annual performance review which will include an assessment of job performance, competence in the role, adherence to company policies and code of conduct, and achievement of role-specific objectives.",4,2024
5,"Information security leaders and managers shall ensure appropriate professional development occurs to provide an understanding of current threats and trends in the security landscape. Security leaders and key stakeholders shall attend trainings, obtain and maintain relevant certifications, and maintain memberships in industry groups as appropriate.",11,2024
6,All <Company Name> employees and third-parties with administrative or privileged technical access to <Company Name> production systems and networks shall complete security awareness training at the time of hire and annually thereafter. Management shall monitor training completion and shall take appropriate steps to ensure compliance with this policy. Employees and contractors shall be aware of relevant information security and data privacy policies and procedures. The company shall ensure that personnel receive security and data privacy training appropriate to their role and data handling responsibilities.,9,2024
7,"In order to maintain a robust level of security awareness, the company will provide security-related updates and communications to company personnel on an on-going basis through multiple communication channels as needed.",10,2024
8,"Background verification checks on <Company Name> personnel shall be carried out in accordance with relevant laws, regulations, and shall be proportional to the business requirements, the classification of the information to be accessed, and the perceived risks. Background screening shall include criminal history checks unless prohibited by local statute. All third-parties with technical privileged or administrative access to <Company Name> production systems or networks are subject to a background check or requirement to provide evidence of an acceptable background, based on their level of access and the perceived risk to <Company Name>.",2,2024
9,"Company policies and information security roles and responsibilities shall be communicated to employees and third-parties at the time of hire or engagement, and employees and contractors are required to formally acknowledge their understanding and acceptance of their security responsibilities. Employees and third-parties with access to company or customer information shall sign an appropriate non-disclosure, confidentiality, and appropriate code-of-conduct agreements. Contractual agreements shall state responsibilities for information security as needed. Employees and relevant third-parties shall follow all <Company Name> information security policies.",5,2024


Yes, we can! Now that all results are fresh, the outdated counterparts were removed from the result set. This way we can supply a nice and clean context to the generation model.

It is also important to see that the same weights produce relevant results for queries about maternity leave—a topic only discussed in the older HR document. It does not get overruled by recent but not relevant documents.

In [15]:
normal_recency_result = app.query(
    knowledgebase_query,
    relevance_weight=1,
    recency_weight=0.25,
    search_query="What are the companies terms for maternal leave?",
    limit=TOP_N,
)

maternity_result = present_result(normal_recency_result)
maternity_result

,body,id,year_created
0,[Company Name] is committed to supporting employees during significant life events such as the birth or adoption of a child. Our Maternity Leave Policy aims to provide eligible employees with the necessary time off to bond with their newborn or newly adopted child and to ensure a smooth transition back to work.,22,2023
1,"Following childbirth or adoption, employees are entitled to [XX weeks/months] of paid maternity leave to recover from childbirth, bond with their newborn or adopted child, and adjust to their new family dynamic. This period of leave is intended to promote the physical and emotional well-being of both the parent and child during the crucial early stages of development.",26,2023
2,"[Company Name] is committed to maintaining the confidentiality of employees' maternity leave and pregnancy-related information in accordance with applicable privacy laws. Additionally, [Company Name] prohibits discrimination or retaliation against employees for exercising their rights under this policy or applicable law.",30,2023
3,"Eligible employees are entitled to up to [XX weeks/months] of paid maternity leave, which includes both prenatal and postnatal leave. Additionally, eligible employees may also take up to [XX weeks/months] of unpaid maternity leave following the conclusion of their paid leave, as provided under the Family and Medical Leave Act (FMLA) and applicable state laws.",24,2023
4,"[Company Name] recognizes the importance of work-life balance and supports flexible work arrangements, including telecommuting and adjusted work schedules, to facilitate a smooth transition back to work for new parents. Employees returning from maternity leave are encouraged to discuss their individual needs and preferences with their supervisor and HR department to explore available options.",27,2023
5,"All full-time and part-time employees are eligible for maternity leave if they have been employed by [Company Name] for at least 12 months and have worked a minimum of 1,250 hours during the 12-month period preceding the commencement of leave. This policy applies equally to biological mothers, adoptive parents, and employees expecting the birth of a child through surrogacy.",23,2023
6,"At the conclusion of maternity leave, employees are expected to return to their regular duties and responsibilities. [Company Name] will make reasonable efforts to accommodate the transition back to work, including providing any necessary training or support to help employees reintegrate into the workplace successfully.",29,2023
7,"Employee and contractor termination and offboarding processes shall ensure that physical and logical access is promptly revoked in accordance with company SLAs and policies, and that all company issued equipment is returned. Any security or confidentiality agreements which remain valid after termination shall be communicated to the employee or contractor at time of termination.",12,2024
8,"This policy applies to all employees of <Company Name>, consultants, contractors and other third-party entities with access to <Company Name> production networks and system resources.",1,2024
9,This Maternity Leave Policy will be reviewed periodically by the HR department to ensure compliance with relevant laws and regulations and to assess its effectiveness in supporting the needs of employees and their families.,32,2023


## Augmentation - formulate query for our LLM for generation

To keep this light and tool independent, we just manually craft our query template based on [LLama2 instructions from HuggingFace](https://huggingface.co/blog/llama2)

In [16]:
context_items_from_retrieval: int = 5
context_text: str = (
    "\n"
    + "\n".join(
        f for f in norm_recency_result_df["body"].iloc[:context_items_from_retrieval]
    )
    + "\n"
)

query = f"""<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>

Please answer the following question by using information from the provided context information!
CONTEXT_INFORMATION: {context_text}
QUESTION: {initial_query_text}
[/INST]"""

In [17]:
print(query)

<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>

Please answer the following question by using information from the provided context information!
CONTEXT_INFORMATION: 
Management shall be responsible for ensuring that information security policies and procedures are reviewed annually, distributed and available, and that employees and contractors abide by those policies and procedures for the duration of their employment or engagement. Annual policy review shall include a review of any linked or referenced p

## Generation - prompt the LLM for text generation to answer our query

In [18]:
# load the 7B parameter Llama model for chat from Huggingface - for the first run this will download model, for subsequent ones it will load it
model = "meta-llama/Llama-2-7b-chat-hf"

# we use the pure transformers library without popular wrappers - but LLamaIndex and LangChain can easily be plugged in here as those use transformers under the hood too
tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [19]:
# we prompt the LLM for text generation with our RAG query
sequences = pipeline(
    query,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Result: <s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>

Please answer the following question by using information from the provided context information!
CONTEXT_INFORMATION: 
Management shall be responsible for ensuring that information security policies and procedures are reviewed annually, distributed and available, and that employees and contractors abide by those policies and procedures for the duration of their employment or engagement. Annual policy review shall include a review of any linked or refe

## Evaluation of the answer

The results are structured, contain the information present in the context documents, and are tailored to our question. Based on hardware, it will run in a manageable time. 
In 3), we can see that the generated text contains the correct annual term. This is mainly because it was fed the right thing as context information.